Lecture 32: Convolutional Autoencoder for Representation Learning
===

In [ ]:
%matplotlib inline
import torch
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.autograd import Function
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy
import time

Load Data:
==

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])
BatchSize = 2000

trainset = torchvision.datasets.CIFAR10(root='./CIFAR10', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BatchSize,
                                          shuffle=True, num_workers=4) # Creating dataloader

testset = torchvision.datasets.CIFAR10(root='./CIFAR10', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BatchSize,
                                         shuffle=False, num_workers=4) # Creating dataloader

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# Check availability of GPU
use_gpu = torch.cuda.is_available()
if use_gpu:
    print('GPU is available!')

Convolutional Autoencoder:
==

In [ ]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.conv_encoder = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.1),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.1),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.1))
        
        self.fc_encoder = nn.Sequential(
            nn.Linear(128*4*4,1024),
            nn.LeakyReLU(0.1))
        
        self.fc_decoder = nn.Sequential(
            nn.Linear(1024,128*4*4),
            nn.LeakyReLU(0.1))
        
        self.conv_decoder = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.1),
            nn.Upsample(scale_factor=2, mode='bilinear'),
            nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.1),
            nn.Upsample(scale_factor=2, mode='bilinear'),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.1),
            nn.Upsample(scale_factor=2, mode='bilinear'),
            nn.Conv2d(in_channels=64, out_channels=3, kernel_size=3, stride=1, padding=1),
            nn.ReLU())

    def forward(self, x):
        x = self.conv_encoder(x)
        x = x.view(-1, 128*4*4)
        x = self.fc_encoder(x)
        x = self.fc_decoder(x)
        x = x.view(-1, 128,4,4)
        x = self.conv_decoder(x)
        return x
net = autoencoder()
if use_gpu:
    net = net.cuda()
print(net)
init_weights = copy.deepcopy(net.conv_encoder[0].weight.data)

Train Autoencoder:
==

In [ ]:
iterations = 10
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3)

trainLoss = []
for epoch in range(iterations):  # loop over the dataset multiple times
    epochStart = time.time()
    runningLoss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        # wrap them in Variable
        if use_gpu:
            inputs = Variable(inputs).cuda()
        else:
            inputs = Variable(inputs)

        optimizer.zero_grad()  # zeroes the gradient buffers of all parameters
        outputs = net(inputs) # forward 
        loss = criterion(outputs, inputs) # calculate loss
        loss.backward() #  backpropagate the loss
        
        optimizer.step()
        runningLoss += loss.data[0]
    trainLoss.append((runningLoss/(60000/BatchSize)))
    epochEnd = time.time()-epochStart
    print('Iteration: {:.0f} /{:.0f}  ;  Training Loss: {:.6f} ; Time consumed: {:.0f}m {:.0f}s '\
          .format(epoch + 1,iterations,runningLoss/(60000/BatchSize),epochEnd//60,epochEnd%60))   
print('Finished Training')

fig = plt.figure()        
plt.plot(range(epoch+1),trainLoss,'g-',label='Loss')
plt.legend(loc='best')
plt.xlabel('Epochs')
plt.ylabel('Training loss') 

Weights Visualization:
====

In [ ]:
# functions to show an image
def imshow(img, strlabel):
    npimg = img.numpy()
    npimg = np.abs(npimg)
    fig_size = plt.rcParams["figure.figsize"]
    fig_size[0] = 10
    fig_size[1] = 10
    plt.rcParams["figure.figsize"] = fig_size
    plt.figure()
    plt.title(strlabel)
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [ ]:
trained_weights = copy.deepcopy(net.conv_encoder[0].weight.data)
d_weights = init_weights - trained_weights 

if use_gpu:
    init_weights = init_weights.view(64,3,3,3).cpu()
    trained_weights = trained_weights.view(64,3,3,3).cpu()
    d_weights = d_weights.view(64,3,3,3).cpu()
else:
    init_weights = init_weights.view(64,3,3,3)
    trained_weights = trained_weights.view(64,3,3,3)
    d_weights = d_weights.view(64,3,3,3)

imshow(torchvision.utils.make_grid(init_weights,nrow=8,normalize=True),'Initial Weights')
imshow(torchvision.utils.make_grid(trained_weights,nrow=8,normalize=True),'Trained Weights')
imshow(torchvision.utils.make_grid(d_weights,nrow=8,normalize=True), 'Weight update')

Modifying the autoencoder for classification:
==

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv = net.conv_encoder
        self.fc1 = net.fc_encoder
        self.fc2 = nn.Sequential(nn.Linear(1024, 10))

    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, 128*4*4)
        x = self.fc1(x)
        x = self.fc2(x)
        return x
net = Model()
print(net)
if use_gpu:
    net = net.cuda()
else:
    net = net

# Copying initial weights  for visualization
cll_weights = copy.deepcopy(net.conv[0].weight.data)

Train Classifier:
==

In [ ]:
iterations = 10
optimizer = optim.Adam(net.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
trainLoss = []
testacc = []

for epoch in range(iterations):  # loop over the dataset multiple times
    epochStart = time.time()
    runningLoss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        
        # wrap them in Variable
        if use_gpu:
            inputs, labels = Variable(inputs).cuda(), Variable(labels).cuda()
        else:
            inputs, labels = Variable(inputs), Variable(labels)

        optimizer.zero_grad()  # zeroes the gradient buffers of all parameters
        outputs = net(inputs) # forward 
        loss = criterion(outputs, labels) # calculate loss
        loss.backward() #  backpropagate the loss
        optimizer.step()
        
        runningLoss += loss.data[0]
        correct = 0
        total = 0
    for data in testloader:
        inputs, labels = data
        if use_gpu:
            inputs, labels = Variable(inputs).cuda(), labels.cuda()
        else:
            inputs, labels = Variable(inputs), labels
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    trainLoss.append((runningLoss/(60000/BatchSize)))
    testacc.append(100 * correct /float(total))
    epochEnd = time.time()-epochStart
    print('Iteration: {:.0f} /{:.0f}  ;  Training Loss: {:.6f} ; Testing Acc: {:.3f} ; Time consumed: {:.0f}m {:.0f}s '\
          .format(epoch + 1,iterations,runningLoss/(60000/BatchSize),100 * correct /float(total),epochEnd//60,epochEnd%60))

print('Finished Training')

fig = plt.figure()        
plt.plot(range(epoch+1),trainLoss,'g-',label='Train Loss')
plt.legend(loc='best')
plt.xlabel('Epochs')
plt.ylabel('Training loss') 

fig = plt.figure()        
plt.plot(range(epoch+1),testacc,'r-',label='Test Acc')
plt.legend(loc='best')
plt.xlabel('Epochs')
plt.ylabel('Test Accuracy') 

Encoder Weights Visualization:
==

In [ ]:
cll_weights_ft = copy.deepcopy(net.conv[0].weight.data)
d_weights = cll_weights-cll_weights_ft 

if use_gpu:
    cll_weights = cll_weights.view(64,3,3,3).cpu()
    cll_weights_ft = cll_weights_ft.view(64,3,3,3).cpu()
    d_weights = d_weights.view(64,3,3,3).cpu()
else:
    cll_weights = cll_weights.view((64,3,3,3))
    cll_weights_ft = cll_weights_ft.view((64,3,3,3))
    d_weights = d_weights.view((64,3,3,3))

imshow(torchvision.utils.make_grid(cll_weights,nrow=8,normalize=True),'Trained Weights')
imshow(torchvision.utils.make_grid(cll_weights_ft,nrow=8,normalize=True),'Finetuned Weights')
imshow(torchvision.utils.make_grid(d_weights,nrow=8,normalize=True), 'Weight update')

Performance of different Classes:
==

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
for data in testloader:
    images, labels = data
    if use_gpu:
        outputs = net(Variable(images.cuda()))
        _, predicted = torch.max(outputs.data.cpu(), 1)
    else:
        outputs = net(Variable(images))
        _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    for i in range(BatchSize):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %f %%' % (
        classes[i], 100 * class_correct[i] / float(class_total[i])))